<a href="https://colab.research.google.com/github/ChiroDeniro/ai-ml-projecten/blob/main/RAG_filosofie_papers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Install packages
!pip install -q transformers accelerate bitsandbytes
!pip install -q langchain langchain-community
!pip install -q chromadb sentence-transformers
!pip install -q pypdf python-dotenv



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 130.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Data Loading

In [10]:
from google.colab import drive
import os
drive.mount('/content/drive')

PDF_FOLDER = '/content/drive/MyDrive/AI_Data/PhilosophyPapers'
print("PDF folder:", PDF_FOLDER)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
PDF folder: /content/drive/MyDrive/AI_Data/PhilosophyPapers


PDF laden en verwerken

In [11]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load all PDFs from your folder
docs = []

for filename in os.listdir(PDF_FOLDER):
    if filename.endswith('.pdf'):
        filepath = os.path.join(PDF_FOLDER, filename)
        loader = PyPDFLoader(filepath)
        docs.extend(loader.load())
        print(f"✓ Loaded: {filename}")

print(f"\n📊 Total: {len(docs)} pages loaded")

✓ Loaded: Exploring Cultural Influences on Neural Foundations of Morality - A Case Against WEIRD Populations.pdf
✓ Loaded: Contemporary_Philosophy.pdf

📊 Total: 10 pages loaded


Documenten Chuncken voor betere verwerking

In [12]:
# Split documents into chunks for better retrieval
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

splits = text_splitter.split_documents(docs)
print(f"✓ Created {len(splits)} chunks from {len(docs)} pages")

✓ Created 30 chunks from 10 pages


Embeddings en Vectors - zoekbare databse creeeren

In [13]:
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory="./philosophy_db"
)

print("✓ Vector database created")


/tmp/ipython-input-2555624108.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✓ Vector database created


Laad OpenSource LLM

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

#Kies model
model_name = "microsoft/Phi-3-mini-4k-instruct"
#model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("✓ Phi-3 loaded")


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

✓ Phi-3 loaded


In [18]:
# Maak de retriever van de Chroma vectorstore
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}   # aantal stukken context
)

print("✓ Retriever klaar")


✓ Retriever klaar


RAG Answer function

In [19]:
def rag_answer(question):
    # RAG context retrieval
    docs = retriever.get_relevant_documents(question)
    context = "\n\n".join([d.page_content for d in docs])

    prompt = f"""
You are a philosophy expert AI.
Answer ONLY using the context below.
If the answer is not in the context, say "I cannot find this in the documents."

Context:
{context}

Question: {question}

Answer:
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.1
    )

    answer = tokenizer.decode(output[0], skip_special_tokens=True)

    # Remove prompt echo
    if "Answer:" in answer:
        answer = answer.split("Answer:")[-1].strip()

    return answer


Chat Gradio

In [26]:
import gradio as gr

def chat_fn(message, history):
    return rag_answer(message)

chatbot = gr.ChatInterface(
    fn=chat_fn,
    title="Filosofie RAG Chatbot",
    description="Stel vragen over je eigen PDF-kennisbank."
)

chatbot.launch()


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://39cc718bb4b04e7e9e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Specifieke Vraag LLM

In [27]:
def test_llm(prompt, max_tokens=500, temperature=0.3):
    print("=== INPUT PROMPT ===")
    print(prompt)
    print("\n=== MODEL OUTPUT ===")

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=temperature,
        top_p=0.95,
        do_sample=True
    )

    text = tokenizer.decode(output[0], skip_special_tokens=True)

    # verwijder prompt echo
    if prompt in text:
        text = text[len(prompt):].strip()

    print(text)
    return text

# Voorbeeld:
test_llm("Leg in detail uit wat Kant bedoelt met de categorische imperatief.", max_tokens=800)


=== INPUT PROMPT ===
Leg in detail uit wat Kant bedoelt met de categorische imperatief.

=== MODEL OUTPUT ===
Kant's categorical imperative is a central philosophical concept in his deontological moral philosophy, which he introduces in his work "Groundwork of the Metaphysics of Morals." The categorical imperative is a principle that one must follow, regardless of desires or consequences, and it is meant to provide a way for individuals to evaluate moral actions and to make moral judgments.

According to Kant, the categorical imperative is categorical because it applies universally and unconditionally. It is not dependent on any particular desires or ends that an individual might have. Instead, it is based on reason and the inherent dignity of rational beings. Kant formulates the categorical imperative in several ways, but the most famous formulation is the "Formula of Universal Law," which states: "Act only according to that maxim whereby you can at the same time will that it should b

'Kant\'s categorical imperative is a central philosophical concept in his deontological moral philosophy, which he introduces in his work "Groundwork of the Metaphysics of Morals." The categorical imperative is a principle that one must follow, regardless of desires or consequences, and it is meant to provide a way for individuals to evaluate moral actions and to make moral judgments.\n\nAccording to Kant, the categorical imperative is categorical because it applies universally and unconditionally. It is not dependent on any particular desires or ends that an individual might have. Instead, it is based on reason and the inherent dignity of rational beings. Kant formulates the categorical imperative in several ways, but the most famous formulation is the "Formula of Universal Law," which states: "Act only according to that maxim whereby you can at the same time will that it should become a universal law."\n\nThis formulation means that one should only act according to principles that co